# Ensemble Learning

## Initial Imports

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [22]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [23]:
# Load the data
file_path = Path('Starter_Code/Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [24]:
# Create our features
X = df.drop(columns = 'loan_status')
X = pd.get_dummies(X)

# Create our target
y = df['loan_status']

In [25]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [26]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk     68470
high_risk      347
Name: count, dtype: int64

In [27]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify=y)

Counter(y_train)

Counter({'low_risk': 51352, 'high_risk': 260})

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [28]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [29]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [30]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [31]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [32]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

np.float64(0.8192848020434227)

In [33]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   62,    25],
       [ 1268, 15850]])

In [34]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.71      0.93      0.09      0.81      0.65        87
   low_risk       1.00      0.93      0.71      0.96      0.81      0.67     17118

avg / total       0.99      0.92      0.71      0.96      0.81      0.67     17205



In [35]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(np.float64(0.06928717515721282), 'total_rec_prncp'),
 (np.float64(0.06195341602973416), 'total_rec_int'),
 (np.float64(0.05999716770508804), 'total_pymnt'),
 (np.float64(0.05944929931774028), 'total_pymnt_inv'),
 (np.float64(0.05780080255994967), 'last_pymnt_amnt'),
 (np.float64(0.02708701665429169), 'int_rate'),
 (np.float64(0.019362938516910073), 'issue_d_Jan-2019'),
 (np.float64(0.01765723264865733), 'dti'),
 (np.float64(0.01635551128868266), 'installment'),
 (np.float64(0.0162869588930354), 'out_prncp'),
 (np.float64(0.016075098739934283), 'annual_inc'),
 (np.float64(0.015822865990250717), 'revol_bal'),
 (np.float64(0.01573225080912679), 'out_prncp_inv'),
 (np.float64(0.015462048804453772), 'max_bal_bc'),
 (np.float64(0.01511218923248116), 'bc_util'),
 (np.float64(0.014818248548877595), 'mths_since_recent_inq'),
 (np.float64(0.014736763773410759), 'mo_sin_old_il_acct'),
 (np.float64(0.014668152866939446), 'avg_cur_bal'),
 (np.float64(0.01454052198931068), 'il_util'),
 (np.float64

### Easy Ensemble Classifier

In [36]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec= EasyEnsembleClassifier(n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, n_jobs=1, random_state=1)

In [37]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

np.float64(0.9056447941469153)

In [38]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   76,    11],
       [ 1066, 16052]])

In [39]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.87      0.94      0.12      0.91      0.81        87
   low_risk       1.00      0.94      0.87      0.97      0.91      0.82     17118

avg / total       0.99      0.94      0.87      0.96      0.91      0.82     17205



### Final Questions

1. Which model had the best balanced accuracy score?

        Easy Ensemble Classifier has the best balanced accuracy score.
        

2. Which model had the best recall score?

        Easy Ensemble Classifier has the best recall score
        

3. Which model had the best geometric mean score?

        Easy Ensemble Classifier has the best geometric mean score
        

4. What are the top three features?

           YOUR ANSWER HERE.